In [1]:
from gensim.models import Word2Vec
 
import pandas as pd

import spacy
import numpy as np
nlp = spacy.load("en_core_web_lg") 

#pyLDAvis specific imports - helps interperate the output of topic models
import pyLDAvis
import pyLDAvis.gensim_models

In [2]:
#Read in and print the csv file
df = pd.read_csv("tf_41178.csv")
print(df.shape)
#print the 39 rows
df.head(39)

(92, 47)


,Date,Type,ID,Name,Body,URL,Role,Title,Background,Comment type,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
0,2020-07-08T01:38:31Z,Commit,8a917da9fad79e74772c8f3bef4ff5f825b88d5c,WindQAQ,Migrate python implementation of gelu,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-08T01:38:42Z,Commit,9d483364d19cd7740c6e93378236f1f9fefb994e,WindQAQ,Port to keras,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-08T01:41:10Z,DESCR,445836524,WindQAQ,As per https://github.com/tensorflow/community...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-07-08T01:48:15Z,PC,655231498,seanpmorgan,Thanks @WindQAQ! Just wanted to add a note...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-07-08T01:55:33Z,Commit,75ad9db9f39709b5ac5dc80a93777df8c4362b55,WindQAQ,run pylint,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-07-08T03:23:32Z,Commit,a3b21999360628de7fc1e553eea4a96c44fd33be,WindQAQ,Update golden api,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-07-08T03:52:42Z,Commit,8adc98973ec85f27d9f362abeb302027f0a86164,WindQAQ,Update golden api,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-07-08T15:20:28Z,RC,444874482,alextp,DISMISSED,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Dismissed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-07-13T20:12:49Z,RC,453904111,karmel,COMMENTED Can we add doc tests with examples s...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-07-13T20:13:06Z,RC,453904243,karmel,COMMENTED Can we add doc tests with examples s...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#New Column: Gives a unique number to each of these categories 
df['category_val'] = df['Comment type'].map({'Conventional review' : 0, 'ML review': 1, 'Dismissed' : 2, 'Management' : 3, 'Other' : 4})
df.head(39)

,Date,Type,ID,Name,Body,URL,Role,Title,Background,Comment type,...,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,category_val
0,2020-07-08T01:38:31Z,Commit,8a917da9fad79e74772c8f3bef4ff5f825b88d5c,WindQAQ,Migrate python implementation of gelu,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2020-07-08T01:38:42Z,Commit,9d483364d19cd7740c6e93378236f1f9fefb994e,WindQAQ,Port to keras,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,2020-07-08T01:41:10Z,DESCR,445836524,WindQAQ,As per https://github.com/tensorflow/community...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,2020-07-08T01:48:15Z,PC,655231498,seanpmorgan,Thanks @WindQAQ! Just wanted to add a note...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,2020-07-08T01:55:33Z,Commit,75ad9db9f39709b5ac5dc80a93777df8c4362b55,WindQAQ,run pylint,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,2020-07-08T03:23:32Z,Commit,a3b21999360628de7fc1e553eea4a96c44fd33be,WindQAQ,Update golden api,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,2020-07-08T03:52:42Z,Commit,8adc98973ec85f27d9f362abeb302027f0a86164,WindQAQ,Update golden api,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,2020-07-08T15:20:28Z,RC,444874482,alextp,DISMISSED,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Dismissed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
8,2020-07-13T20:12:49Z,RC,453904111,karmel,COMMENTED Can we add doc tests with examples s...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,2020-07-13T20:13:06Z,RC,453904243,karmel,COMMENTED Can we add doc tests with examples s...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
#Tokenizing and removing stop words from the original input value
#Turning the input value into a numerical vlaue that the model understands
def preprocess_and_vectorize(text):
    #check value of input string
    if not isinstance(text, str):
        return np.zeros((300,))  #return a zero vector if the input is not a string - using 300 since Spacy uses 300-dimentional zero vectors
    #remove stop words and lemmatize the text
    doc = nlp(text)
    vectors = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        vectors.append(token.vector)
    print(vectors)
    
    if vectors:  # check if vectors list is not empty
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros((300,))  # return a zero vector if the vectors list is empty
    return mean_vector

In [5]:
#New Column: Gives. vectorized value of each input value 
df['vector'] = df['Body'].iloc[:39].apply(lambda text: preprocess_and_vectorize(text))
df.head(39)
# df['vector'] = df['Body'].apply(lambda text: preprocess_and_vectorize(text))
# df.head(39)

[array([-2.4752  ,  0.52168 ,  0.16867 ,  3.2994  ,  2.1103  ,  0.25    ,
        0.046571,  2.7635  , -1.3151  ,  1.1516  ,  3.4943  ,  1.0532  ,
       -4.1382  ,  3.3488  ,  2.7279  ,  0.023003,  1.3536  ,  1.4741  ,
       -2.6494  , -2.9689  , -2.6738  , -2.7591  , -1.2577  ,  1.3407  ,
        0.58053 ,  0.32727 , -3.3395  , -0.35698 , -1.2736  ,  1.3963  ,
       -1.476   , -1.4323  , -1.1268  , -0.25453 , -1.9975  , -2.7572  ,
        1.3462  , -1.0832  ,  0.91696 , -3.0349  , -0.76496 , -1.9992  ,
        0.063289,  3.5607  , -2.7648  ,  0.76119 ,  2.2463  , -1.5844  ,
       -2.0389  , -1.5829  ,  1.0624  ,  2.8675  , -2.8235  , -4.5756  ,
       -2.7501  , -0.085979, -5.9145  ,  1.1754  ,  1.1318  , -0.38072 ,
       -3.2722  , -2.1684  , -0.4984  ,  0.88358 ,  5.0679  ,  2.0915  ,
       -2.7009  ,  0.22031 ,  2.2098  , -0.36808 ,  2.1906  , -3.6019  ,
        4.1686  ,  1.5225  , -0.43594 ,  0.42891 , -1.6773  , -0.28204 ,
       -0.59196 ,  1.0297  , -2.7454  , -0.89824 ,

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0

[array([ 5.6293e+00,  3.8344e-01,  3.9918e+00, -4.7544e-03,  7.0913e+00,
       -1.7423e+00,  1.9326e-01, -2.8261e+00, -1.0093e+01,  1.9940e+00,
        3.3710e+00, -2.9693e-01, -3.0977e+00, -1.5225e+00,  6.1597e-01,
       -4.1121e+00,  1.3466e+00, -1.3614e+00, -4.2605e+00, -2.6252e+00,
       -3.5951e+00, -1.9430e+00, -5.5244e-01,  2.1207e+00,  1.1495e+00,
       -4.3356e-01,  5.0239e+00,  2.0360e+00, -2.2927e+00, -3.7541e+00,
       -4.8320e+00,  1.3887e+00, -7.7209e-01, -9.8068e-01,  1.1691e+00,
        4.1606e+00,  5.5331e+00,  1.9647e+00,  1.2600e+00, -7.4863e-01,
       -5.9344e-01, -1.9802e+00, -4.5774e+00, -3.7403e+00,  3.0180e+00,
       -4.4826e-01,  5.3002e+00,  7.8099e+00,  2.6391e+00, -2.6289e+00,
       -1.5084e+00,  7.6542e-01,  5.2254e-01,  1.5460e+00, -2.5084e+00,
        6.3787e+00, -9.2897e+00,  4.5474e+00,  5.1502e+00,  9.5582e-01,
       -3.0571e+00, -1.1054e+00,  1.2254e-01, -9.8187e-01,  1.3821e+00,
        2.8535e+00, -4.2543e+00, -4.5537e+00,  3.5449e+00,  6.7

[array([ -3.5197  ,  -1.7391  ,   6.7862  ,   7.0175  ,  -3.5325  ,
        -2.7952  ,   5.2157  ,  -0.88051 , -14.586   ,   3.5412  ,
         1.7307  ,  -1.8918  , -13.105   ,   3.829   ,  -6.8345  ,
         1.0634  ,   3.1709  ,   0.7697  ,  -7.3413  ,   1.0749  ,
         6.5215  ,  -5.7121  , -11.706   ,   8.3819  , -13.942   ,
        -8.866   ,   8.3244  ,   4.3721  ,   1.0374  ,  -1.9302  ,
        -1.76    ,   2.21    ,  -2.1362  ,  12.166   ,  16.788   ,
        -4.4009  ,  -9.1114  ,  -6.5344  ,   8.2072  ,   1.2667  ,
         8.6758  ,  -3.1011  ,   0.66778 ,   0.82505 ,  -8.7196  ,
         3.2009  ,   4.7736  ,   7.8539  ,  -1.7608  ,  -7.1417  ,
        -5.9454  ,   6.7545  , -13.604   ,   4.1885  ,  -1.9194  ,
        12.072   ,   1.1429  ,   1.7391  ,   0.74174 ,  -0.80123 ,
         2.2964  ,   8.024   ,  -6.26    ,   1.8075  ,  -1.3522  ,
         1.3007  ,  -6.8518  ,  12.147   ,  -5.2141  ,   2.8496  ,
        -9.2725  ,  -3.0841  ,  -1.0329  ,   7.237   ,  -6.51

[array([-1.9632e+00,  6.8090e-01,  3.5039e-01,  2.8093e+00,  2.5849e+00,
       -9.4742e-01,  1.0876e+00, -1.8570e+00, -2.7482e+00,  1.7624e+00,
        2.3628e+00,  7.4662e-02, -3.1649e+00, -1.5335e-03,  1.3470e+00,
        3.8438e-01,  5.5811e+00,  3.6465e-01, -2.7879e+00, -1.1630e-01,
       -1.6139e+00, -1.5642e+00, -9.2785e-01, -1.1433e+00,  2.2389e+00,
       -9.9517e-02, -1.9565e+00, -5.4527e+00, -2.9576e+00,  1.7537e+00,
       -1.2429e+00,  1.2622e+00,  2.5588e+00,  3.7863e+00,  6.8890e-01,
       -4.3934e+00,  1.5935e+00, -9.1783e-01, -1.6133e+00,  3.4175e-01,
        3.5779e+00, -4.7537e+00, -2.0566e+00,  5.1200e+00, -2.6870e+00,
        1.9213e-01,  3.8848e+00, -7.5909e-01,  4.0589e-01, -5.4443e+00,
       -1.4851e+00,  2.9390e-01, -1.9395e+00, -1.6153e+00, -3.6345e+00,
        2.6537e+00, -4.6369e+00, -8.4780e-01, -2.2303e+00, -4.0760e-01,
       -1.7693e-01, -6.3584e-03,  1.6888e-01,  4.1877e+00, -8.1750e-01,
        1.3919e+00, -1.7128e+00, -5.3018e+00,  3.2328e+00,  1.0

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0

,Date,Type,ID,Name,Body,URL,Role,Title,Background,Comment type,...,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,category_val,vector
0,2020-07-08T01:38:31Z,Commit,8a917da9fad79e74772c8f3bef4ff5f825b88d5c,WindQAQ,Migrate python implementation of gelu,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"[-1.3958325, -0.04547, 1.4323175, 0.9047251, 1..."
1,2020-07-08T01:38:42Z,Commit,9d483364d19cd7740c6e93378236f1f9fefb994e,WindQAQ,Port to keras,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"[-2.724385, 0.18097, 4.0846, 1.1163001, -2.781..."
2,2020-07-08T01:41:10Z,DESCR,445836524,WindQAQ,As per https://github.com/tensorflow/community...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"[-0.8761955, -0.062730916, 2.154294, 0.70719, ..."
3,2020-07-08T01:48:15Z,PC,655231498,seanpmorgan,Thanks @WindQAQ! Just wanted to add a note...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[-0.011326067, -0.6489289, -0.10396383, 0.7941..."
4,2020-07-08T01:55:33Z,Commit,75ad9db9f39709b5ac5dc80a93777df8c4362b55,WindQAQ,run pylint,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"[-2.3616, 1.60865, -0.9432, -0.78875, 1.5345, ..."
5,2020-07-08T03:23:32Z,Commit,a3b21999360628de7fc1e553eea4a96c44fd33be,WindQAQ,Update golden api,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"[1.4250201, -1.7152977, 2.8888733, 1.3160866, ..."
6,2020-07-08T03:52:42Z,Commit,8adc98973ec85f27d9f362abeb302027f0a86164,WindQAQ,Update golden api,https://github.com/tensorflow/tensorflow/commi...,NaN,NaN,NaN,ML review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"[1.4250201, -1.7152977, 2.8888733, 1.3160866, ..."
7,2020-07-08T15:20:28Z,RC,444874482,alextp,DISMISSED,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Dismissed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,"[0.053037, -1.4706, -0.72252, -1.9655, -0.9202..."
8,2020-07-13T20:12:49Z,RC,453904111,karmel,COMMENTED Can we add doc tests with examples s...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[-0.38826314, -0.8052316, 0.366797, -0.0815210..."
9,2020-07-13T20:13:06Z,RC,453904243,karmel,COMMENTED Can we add doc tests with examples s...,https://github.com/tensorflow/tensorflow/pull/...,NaN,NaN,NaN,Conventional review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[-0.38826314, -0.8052316, 0.366797, -0.0815210..."
